# Setup

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

# Daten laden

In [10]:
# Define the data loaders
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


# Define the Neural Network

In [11]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

import torch.nn as nn
import torch.nn.functional as F

# Definition des CNN-Modells
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

net = Net()

# Zusammenfassung des Modells
summary(net, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
         MaxPool2d-2           [-1, 32, 16, 16]               0
            Conv2d-3           [-1, 64, 16, 16]          18,496
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5            [-1, 128, 8, 8]          73,856
         MaxPool2d-6            [-1, 128, 4, 4]               0
            Linear-7                  [-1, 512]       1,049,088
           Dropout-8                  [-1, 512]               0
            Linear-9                  [-1, 256]         131,328
          Dropout-10                  [-1, 256]               0
           Linear-11                   [-1, 10]           2,570
Total params: 1,276,234
Trainable params: 1,276,234
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forw

# Sweep Config definieren

In [12]:
# creating sweep config for random forest classifier
sweep_config = {
    'method': 'grid', # grid search over hyperparameters
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'lr': {'values': [0.0001, 0.01, 0.1]},
        'batch_size': {'values': [2, 8, 32]},
    }
}

# Initialize a new wandb sweep
sweep_id = wandb.sweep(sweep_config, project='cifar10-classification')

Create sweep with ID: 2b5f46tj
Sweep URL: https://wandb.ai/7ben18/cifar10-classification/sweeps/2b5f46tj


## Define the Training Loop

In [16]:
# criterion und optimizer definieren
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# device definieren
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



def train_loop(model, criterion, optimizer, train_loader, val_loader, device, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        for i, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * targets.size(0)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        train_loss /= len(train_loader.dataset)
        train_acc = 100.0 * correct / total

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * targets.size(0)
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
        val_loss /= len(val_loader.dataset)
        val_acc = 100.0 * correct / total

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")

        # Log the metrics to Weights & Biases
        wandb.log({"Train Loss": train_loss, "Val Loss": val_loss, "Train Acc": train_acc, "Val Acc": val_acc})

In [17]:
train_loop(net, criterion, optimizer, train_loader, val_loader, device, epochs=10)

KeyboardInterrupt: 